In [31]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook

import json, os


In [32]:
f = open('data/hdxusd_orderbook.json', 'r')
y = json.loads(f.read())
hdxusd_orderbook = y['result']['HDXUSD']

f = open('data/dotusd_orderbook.json', 'r')
y = json.loads(f.read())
dotusd_orderbook = y['result']['DOTUSD']



In [33]:

# dot_usdt_order_book = {
#     'bids': [{'price': 3.60, 'amount': 200},
#              {'price': 3.59, 'amount': 100},
#              {'price': 3.50, 'amount': 100},
#              {'price': 3.40, 'amount': 2000}],
#     'asks': [{'price': 3.70, 'amount': 100},
#              {'price': 3.74, 'amount': 5000},
#              {'price': 3.80, 'amount': 200},
#              {'price': 3.90, 'amount': 2000}]
# }
# 
# hdx_usdt_order_book = {
#     'bids': [{'price': 0.03, 'amount': 2000},
#              {'price': 0.025, 'amount': 2000},
#              {'price': 0.02, 'amount': 2000},
#              {'price': 0.015, 'amount': 2000}],
#     'asks': [{'price': 0.04, 'amount': 2000},
#              {'price': 0.05, 'amount': 2000},
#              {'price': 0.06, 'amount': 2000},
#              {'price': 0.07, 'amount': 2000}]
# }

# hdx_dot_order_book = {
#     'bids': [{'price': 0.005, 'amount': 2000},
#              {'price': 0.004, 'amount': 2000}],
#     'asks': [{'price': 0.0052, 'amount': 2000},
#              {'price': 0.0055, 'amount': 2000}]
# }

dot_usdt_order_book = parse_kraken_orderbook(dotusd_orderbook)
hdx_usdt_order_book = parse_kraken_orderbook(hdxusd_orderbook)

order_book = {
    ('DOT','USDT'): dot_usdt_order_book,
    ('HDX','USDT'): hdx_usdt_order_book,
    # ('HDX','DOT'): hdx_dot_order_book
}

tokens = {
    'USDT': {
        'liquidity': 2062772,
        'LRNA': 2062772
    },
    'DOT': {
        # 'liquidity': 389000,
        'liquidity': 350000,
        'LRNA': 1456248
    },
    'HDX': {
        'liquidity': 108000000,
        'LRNA': 494896
    }
}

lrna_fee = 0.0005
asset_fee = 0.0025
cex_fee = 0.0016

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0

op_state = OmnipoolState(
            tokens=tokens,
            lrna_fee=lrna_fee,
            asset_fee=asset_fee,
    preferred_stablecoin='USDT',
        )

init_state = op_state.copy()

iters = 10

In [34]:
all_swaps = {}
for tkn_pair in order_book:
    pair_order_book = order_book[tkn_pair]
    tkn = tkn_pair[0]
    numeraire = tkn_pair[1]
    bids = pair_order_book['bids']
    asks = pair_order_book['asks']
    bids = sorted(bids, key=lambda x: x['price'], reverse=True)
    asks = sorted(asks, key=lambda x: x['price'], reverse=False)
    
    agent = Agent(holdings={'USDT': 1000000, 'DOT': 1000000, 'HDX': 1000000}, unique_id='bot')
    
    op_spot = OmnipoolState.price(op_state, tkn, numeraire)
    buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
    sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
    i = 0
    profit = 0
    bid_executed = False
    ask_executed = False
    last_profit = 0
    if buy_spot < bids[i]['price'] * (1 - cex_fee):
        bid_executed = True
        test_state = op_state
        test_agent = agent
        while i < len(bids) and buy_spot < bids[i]['price'] * (1 - cex_fee):  # spot is lower than the highest bid, so we should buy from Omnipool and sell to CEX
            print('buy')
            op_state = test_state
            agent = test_agent
            test_state = op_state.copy()
            test_agent = agent.copy()
            test_state.swap(test_agent, tkn_buy=tkn, tkn_sell=numeraire, buy_quantity=bids[i]['amount'])
            op_spot = OmnipoolState.price(test_state, tkn, numeraire)
            buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
            i += 1
    elif sell_spot > asks[i]['price'] / (1 - cex_fee):
        ask_executed = True
        test_state = op_state
        test_agent = agent
        while i < len(bids) and sell_spot > asks[i]['price'] / (1 - cex_fee):  # spot is higher than the lowest ask, so we should buy from CEX and sell to Omnipool
            print('sell')
            op_state = test_state
            agent = test_agent
            test_state = op_state.copy()
            test_agent = agent.copy()
            test_state.swap(test_agent, tkn_buy=numeraire, tkn_sell=tkn, sell_quantity=asks[i]['amount'])
            op_spot = OmnipoolState.price(test_state, tkn, numeraire)
            sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
            i += 1
    
    swaps = []
    if bid_executed:
        if buy_spot < bids[i-1]['price'] * (1 - cex_fee):  # last trade can be fully executed without moving spot price too much
            swaps = [('buy',bids[j]) for j in range(i)]
        else:  # Use binary search to partially fill last order
            swaps = bids[0:i-1]
            swaps = [('buy',bids[j]) for j in range(i-1)]
            bid = bids[i-1]
            amt_high = bid['amount']
            amt_low = 0
            
            for i in range(iters):
                test_agent = agent.copy()
                test_state = op_state.copy()
                amt = amt_low + (amt_high - amt_low) / 2
                test_state.swap(test_agent, tkn_buy=tkn, tkn_sell=numeraire, buy_quantity=amt)
                op_spot = OmnipoolState.price(test_state, tkn, numeraire)
                buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
                if buy_spot > bid['price'] * (1 - cex_fee):
                    amt_high = amt
                elif buy_spot < bid['price'] * (1 - cex_fee):
                    amt_low = amt
                else:
                    break
                    
            op_state.swap(agent, tkn_buy='DOT', tkn_sell=numeraire, buy_quantity=amt_low)
            op_spot = OmnipoolState.price(op_state, tkn, numeraire)
            swaps.append(('buy',{'price': bid['price'], 'amount': amt_low}))

    
    elif ask_executed:
        if sell_spot > asks[i-1]['price'] / (1 - cex_fee):  # last trade can be fully executed without moving spot price too much
            swaps = [('sell',asks[j]) for j in range(i)]
        else:  # Use binary search to partially fill last order
            swaps = [('sell',asks[j]) for j in range(i-1)]
            ask = asks[i-1]
            amt_high = ask['amount']
            amt_low = 0
            for i in range(iters):
                test_agent = agent.copy()
                test_state = op_state.copy()
                amt = amt_low + (amt_high - amt_low) / 2
                test_state.swap(test_agent, tkn_buy=numeraire, tkn_sell=tkn, sell_quantity=amt)
                op_spot = OmnipoolState.price(test_state, tkn, numeraire)
                sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
                if sell_spot < ask['price'] / (1 - cex_fee):
                    amt_high = amt
                elif sell_spot > ask['price'] / (1 - cex_fee):
                    amt_low = amt
                else:
                    break
                    
            op_state.swap(agent, tkn_buy=numeraire, tkn_sell=tkn, buy_quantity=amt_low)
            op_spot = OmnipoolState.price(op_state, tkn, numeraire)
            swaps.append(('sell',{'price': ask['price'], 'amount': amt_low}))


    print(op_spot)
    all_swaps[tkn_pair] = swaps
print(all_swaps)

sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
3.7876558182200677
0.004410362380802018
{('DOT', 'USDT'): [('sell', {'price': 3.7398, 'amount': 14.061}), ('sell', {'price': 3.7406, 'amount': 93.629}), ('sell', {'price': 3.7407, 'amount': 43.0}), ('sell', {'price': 3.7408, 'amount': 40.0}), ('sell', {'price': 3.741, 'amount': 390.018}), ('sell', {'price': 3.7417, 'amount': 3957.426}), ('sell', {'price': 3.7418, 'amount': 3334.869}), ('sell', {'price': 3.7429, 'amount': 374.508}), ('sell', {'price': 3.743, 'amount': 1347.0}), ('sell', {'price': 3.7432, 'amount': 1054.44140625})], ('HDX', 'USDT'): []}


In [35]:
state = init_state.copy()
init_agent = Agent(holdings={'USDT': 1000000, 'DOT': 1000000, 'HDX': 1000000}, unique_id='bot')
agent = init_agent.copy()
for tkn_pair in all_swaps:
    swaps = all_swaps[tkn_pair]
    
    for swap in swaps:
        
        if swap[0] == 'buy':
            # omnipool leg
            state.swap(agent, tkn_buy=tkn_pair[0], tkn_sell=tkn_pair[1], buy_quantity=swap[1]['amount'])
            # CEX leg
            agent.holdings[tkn_pair[0]] -= swap[1]['amount']
            agent.holdings[tkn_pair[1]] += swap[1]['amount'] * swap[1]['price']
        elif swap[0] == 'sell':
            # omnipool leg
            state.swap(agent, tkn_buy=tkn_pair[1], tkn_sell=tkn_pair[0], sell_quantity=swap[1]['amount'])
            # CEX leg
            agent.holdings[tkn_pair[0]] += swap[1]['amount']
            agent.holdings[tkn_pair[1]] -= swap[1]['amount'] * swap[1]['price']

profit = {tkn: agent.holdings[tkn] - init_agent.holdings[tkn] for tkn in agent.holdings}
print(profit)
            
for tkn_pair in all_swaps:
    print(tkn_pair)
    op_spot = OmnipoolState.price(state, tkn_pair[0], tkn_pair[1])
    buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
    sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
    print((sell_spot, op_spot, buy_spot))
    


{'USDT': 2146.992296691402, 'DOT': 0.0, 'HDX': 0}
('DOT', 'USDT')
(3.7492229336233476, 3.760499732195268, 3.771810448778542)
('HDX', 'USDT')
(0.004384461725138867, 0.004397649175604209, 0.00441087629088149)
